In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
import plotly.graph_objects as go

import warnings 
warnings.filterwarnings('ignore')

In [2]:
start = datetime(2014, 9, 17)
end = datetime.now().date().isoformat()
symbol = 'BTC-USD'


In [3]:
data = yf.download(symbol, start = start, end = end)

[*********************100%%**********************]  1 of 1 completed


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3476 entries, 2014-09-17 to 2024-03-23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3476 non-null   float64
 1   High       3476 non-null   float64
 2   Low        3476 non-null   float64
 3   Close      3476 non-null   float64
 4   Adj Close  3476 non-null   float64
 5   Volume     3476 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 190.1 KB


In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [6]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-19,67556.132812,68106.929688,61536.179688,61912.773438,61912.773438,74215844794
2024-03-20,61930.156250,68115.257812,60807.785156,67913.671875,67913.671875,66792634382
2024-03-21,67911.585938,68199.992188,64580.917969,65491.390625,65491.390625,44480350565
2024-03-22,65489.929688,66623.750000,62355.371094,63778.761719,63778.761719,41401116964
2024-03-23,63802.722656,65976.398438,63038.492188,64062.203125,64062.203125,24738964812


In [7]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3476.000000,3476.000000,3476.000000,3476.000000,3476.000000,3.476000e+03
mean,15459.888963,15822.713012,15074.012036,15476.923371,15476.923371,1.697648e+10
std,17045.585359,17459.619696,16592.473470,17061.009028,17061.009028,1.927613e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,989.725235,1008.620026,956.867020,990.237274,990.237274,1.971030e+08
50%,8573.910156,8740.240234,8295.844238,8582.224609,8582.224609,1.246005e+10
75%,26330.956055,26758.425781,25933.351562,26338.659180,26338.659180,2.727900e+10
max,73079.375000,73750.070312,71334.093750,73083.500000,73083.500000,3.509679e+11


In [8]:
#Null Values Check
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [9]:
data.shape

(3476, 6)

In [10]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [11]:
fig = px.line(data.Close)
fig.show()

In [12]:
data['ma100'] = data['Close'].rolling(window=100).mean()
fig2 = px.line(data, x=data.index, y=['Close', 'ma100'], color_discrete_map={'Close': 'blue', 'ma100': 'red'})
fig2.show()

In [13]:
data['ma200'] = data['Close'].rolling(window=200).mean()
fig3 = px.line(data, x=data.index, y=['Close', 'ma200'], color_discrete_map={'Close': 'blue', 'ma200': 'red'})
fig3.show()

In [14]:
fig4 = px.line(data, x=data.index, y=['Close', 'ma100', 'ma200'], color_discrete_map={'Close': 'blue', 'ma100':'yellow', 'ma200': 'red'})
fig4.show()

In [15]:
# Prepare Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [16]:
prediction_days = 80
future_day = 15

In [17]:
x_train, y_train = [], []

In [18]:
for x in range(prediction_days, len(scaled_data)-future_day):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x+future_day, 0])

In [19]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [20]:
# Create Neural Network
model = Sequential()

# Layer 1
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Layer 2
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

# Layer 3
model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 80, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 80, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 80, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,851 (198.64 KB)

 Trainable params: 50,851 (198.64 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50, batch_size=32)

Epoch 1/50


106/106 ━━━━━━━━━━━━━━━━━━━━ 27s 149ms/step - loss: 0.0181
Epoch 2/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 17s 160ms/step - loss: 0.0050
Epoch 3/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 22s 212ms/step - loss: 0.0043
Epoch 4/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 19s 178ms/step - loss: 0.0037
Epoch 5/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 18s 165ms/step - loss: 0.0042
Epoch 6/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 17s 159ms/step - loss: 0.0034
Epoch 7/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 16s 155ms/step - loss: 0.0034
Epoch 8/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 16s 153ms/step - loss: 0.0033
Epoch 9/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 17s 158ms/step - loss: 0.0037
Epoch 10/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 16s 152ms/step - loss: 0.0031
Epoch 11/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 16s 155ms/step - loss: 0.0035
Epoch 12/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 17s 157ms/step - loss: 0.0028
Epoch 13/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 16s 155ms/step - loss: 0.0031
Epoch 14/50
106/106 ━━━━━━━━━━━━━━━━━━━━ 17s 158ms/step - loss: 0.0034
Epoch 15/50
106/106 ━━━━━━

In [44]:
model.save('my_model.keras')

In [45]:
start = datetime(2020, 1, 1)
end = datetime.now().date().isoformat()
symbol = 'BTC-USD'
test_data = yf.download(symbol, start = start, end = end)

[*********************100%%**********************]  1 of 1 completed


In [46]:
test_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [47]:
test_data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-03-19,67556.132812,68106.929688,61536.179688,61912.773438,61912.773438,74215844794
2024-03-20,61930.156250,68115.257812,60807.785156,67913.671875,67913.671875,66792634382
2024-03-21,67911.585938,68199.992188,64580.917969,65491.390625,65491.390625,44480350565
2024-03-22,65489.929688,66623.750000,62355.371094,63778.761719,63778.761719,41401116964
2024-03-23,63802.722656,65976.398438,63038.492188,64062.203125,64062.203125,24738964812


In [48]:
actual_prices = test_data['Close'].values

In [49]:
total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

In [50]:
model_inputs = total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.fit_transform(model_inputs)

In [51]:
x_test = []

In [52]:
for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

In [53]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [54]:
prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

 2/49 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step 

49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step


In [55]:
actual_prices.shape


(1544,)

In [56]:
prediction_prices.shape

(1544, 1)

In [57]:
# Creating a Plotly figure
fig = go.Figure()

# Trace for actual prices
fig.add_trace(go.Scatter(x=test_data.index, y=actual_prices.flatten(), mode='lines', name='Actual Prices', line=dict(color='blue')))

# Trace for predicted prices
fig.add_trace(go.Scatter(x=test_data.index, y=prediction_prices.flatten(), mode='lines', name='Predicted Prices', line=dict(color='red')))

# Update layout
fig.update_layout(
                  xaxis_title='Date',
                  yaxis_title='Price',
                  legend=dict(x=0, y=1.1, orientation='h'),
                  margin=dict(l=0, r=0, t=50, b=0))

# Displaying the plot
fig.show()

In [58]:
# Predict Next Day
real_data = [model_inputs[len(model_inputs)+1-prediction_days:len(model_inputs)+1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

In [59]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [60]:
print(end)

2024-03-24


In [61]:
print(prediction)

[[58965.477]]


In [62]:
test_data['Close'].mean()

30133.655559954866

In [63]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Mean Absolute Error (MAE)
mae = mean_absolute_error(actual_prices, prediction_prices)

# Mean Squared Error (MSE)
mse = mean_squared_error(actual_prices, prediction_prices)

# Root Mean Squared Error (RMSE)
rmse = mean_squared_error(actual_prices, prediction_prices, squared=False)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')

Mean Absolute Error (MAE): 2096.63
Mean Squared Error (MSE): 17929362.84
Root Mean Squared Error (RMSE): 4234.31
